In [1]:
pip install numpy pandas tensorflow scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import ast
import json

In [2]:

# Load your data (replace with your file path)
#data= pd.read_parquet('your_landmark_data.parquet')

df_from_parquet = pd.read_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks_lh_rh")


In [3]:

unique_labels = df_from_parquet['Label'].unique()
label_id_mapping = {label: idx for idx, label in enumerate(unique_labels)}

# Step 3: Map the Label to Label_id in the DataFrame
df_from_parquet['Label_id'] = df_from_parquet['Label'].map(label_id_mapping)
label_info = [{'Label': label, 'Label_id': label_id} for label, label_id in label_id_mapping.items()]


# Step 4: Ensure Keypoints are converted to lists if they are not already
df_from_parquet['Keypoints'] = df_from_parquet['Keypoints'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

# Step 5: Group by the necessary columns and aggregate
grouped = df_from_parquet.groupby(['Label','Label_id', 'sample_type', 'Sample_Number']).apply(
    lambda x: x[['Frame_Number', 'Keypoints']].to_dict('records')
).reset_index(name='frames_keypoints')

# Step 6: Convert the grouped result to a JSON-like structure
grouped_json = grouped.to_dict(orient='records')

# Step 7: Convert the result to a JSON string
json_output = json.dumps(grouped_json, indent=4)


In [ ]:
data = json.loads(json_output)
print(data)

In [ ]:
data[1]

In [4]:
import pandas as pd
import json

# Read from a parquet file

# Proceed with mapping and processing as before
unique_labels = df_from_parquet['Label'].unique()
label_id_mapping = {label: idx for idx, label in enumerate(unique_labels)}

df_from_parquet['Label_id'] = df_from_parquet['Label'].map(label_id_mapping)
df_from_parquet['Keypoints'] = df_from_parquet['Keypoints'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

# Ensure this is a DataFrame before passing it
if isinstance(df_from_parquet, pd.DataFrame):
    grouped = df_from_parquet.groupby(['Label', 'Label_id', 'sample_type', 'Sample_Number']).apply(
        lambda x: x[['Frame_Number', 'Keypoints']].to_dict('records')
    ).reset_index(name='frames_keypoints')
    
    grouped_json = grouped.to_dict(orient='records')
    json_output = json.dumps(grouped_json, indent=4)

    # Optionally save to a file
    with open('output.json', 'w') as json_file:
        json_file.write(json_output)
else:
    print("Data is not a DataFrame.")


In [ ]:
data = json.loads(json_output)
print(data[0])

In [25]:
import re
import numpy as np

def extract_keypoints(df):
    keypoints = []
    labels = []
    
    for _, row in df.iterrows():
        keypoint_str = row[1]['frames_keypoints'][0]['Keypoints']
        
        if isinstance(keypoint_str, str):
            print("Is Str")
            # Remove curly braces and split the string by commas
            keypoint_str = keypoint_str.replace('{', '').replace('}', '')
            try:
                print("Is not Str")
                # Convert the string into a list of floats
                keypoint_list = [float(val) for val in keypoint_str.split(',')]
                keypoint_str = keypoint_list.replace('{', '').replace('}', '')
            except ValueError as e:
                print(f"Error converting keypoints: {keypoint_str} -> {e}")
                continue  # Skip problematic rows if conversion fails
        else:
            keypoint_list = keypoint_str  # If not a string, assume it's already correct
        
        keypoints.append(keypoint_list)
        labels.append(row['Label'])  # Assuming label is numeric
    
    return np.array(keypoints), np.array(labels)


In [ ]:
data.iterrows()

In [ ]:
data[0]
#['frames_keypoints'][0]['Keypoints']

In [ ]:
X, y = extract_keypoints(data)


In [ ]:

# Check keypoints data and ensure it's numeric
print("Keypoints shape: ", X.shape)
print("Sample keypoints data: ", X[0])



In [ ]:
X = X.astype(np.float32)  # Convert keypoints to float32
X = X / np.max(X)  # Normalize between 0 and 1
